<a href="https://colab.research.google.com/github/carlosjesus23/LLMs/blob/main/gsi073_aula0_regressao_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch
import torch.nn as nn
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X_np = iris.data
y_np = (iris.target == 1).astype(float)

# 2. Preparar dados para pytorch
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# 2.1. Separar dados em Treino e Teste (Esta linha retorna ARRAYS DO NUMPY)
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_np, y_np, test_size=0.2, random_state=42
)

# 2.2. NOVO PASSO: Converter os arrays do numpy de volta para tensores PyTorch
X_train = torch.tensor(X_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test_np, dtype=torch.float32).view(-1, 1)

## Definição do modelo e treinamento

In [ ]:
# 3. Definir modelo: regressão logística
modelo = nn.Linear(4, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)

# 4. Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.1)

## Execução do treinamento

In [ ]:
# 5. Treino
for epoch in range(1000):
    optimizer.zero_grad() # reseta gradiente senão acumula
    outputs = modelo(X_train)
    loss = funcao_perda(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

## Execução da Avaliação do Modelo

In [ ]:
# 6. Avaliação do Modelo
with torch.no_grad():
    test_logits = modelo(X_test)
    test_probabilities = torch.sigmoid(test_logits)
    test_predictions = (test_probabilities >= 0.5).float()

    # Converter para numpy
    y_true = y_test.numpy().flatten()
    y_pred = test_predictions.numpy().flatten()

    # Calcular métricas
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print("\n📊 Métricas no conjunto de teste:")
    print(f"✅ Acurácia : {accuracy * 100:.2f}%")
    print(f"🎯 Precisão : {precision * 100:.2f}%")
    print(f"🔍 Recall   : {recall * 100:.2f}%")
    print(f"⚖️ F1-Score : {f1 * 100:.2f}%")